In [1]:
import os
import pandas as pd
import datetime
import json
import re
import tushare as ts
import gensim
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

C:\Users\jgtzsx01\AppData\Local\Continuum\Anaconda2\lib\site-packages\gensim\utils.py:855: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [33]:
import matplotlib as mpl
mpl.rcParams['font.sans-serif'] = ['Microsoft YaHei'] #指定默认字体  
mpl.rcParams['axes.unicode_minus'] = False # 解决保存图像是负号'-'显示为方块的问题
# sns.axes_style()，可以看到是否成功设定字体为微软雅黑。

In [2]:
sns.set_style("whitegrid")
sns.set_context("talk")

In [90]:
data_dir = "C:/Users/jgtzsx01/Documents/workspace/data/sina"
csv_dir = "%s/day-news"%(data_dir)
key_words_dir = "%s/week-keywords"%(data_dir)
word_count_file = "%s/word_count.json"%(data_dir)
asset_class_dir = "C:/Users/jgtzsx01/Documents/workspace/data/asset-class"

In [203]:
# model = gensim.models.Word2Vec.load("C:/Users/jgtzsx01/Documents/workspace/model/model")
model = gensim.models.Word2Vec.load("C:/Users/jgtzsx01/Documents/workspace/model/wallstreet_model")

In [204]:
len(model.vocab)

84956

In [205]:
# 判断是否是中文
def isChinese(word):
    for w in word:
        if re.match('[ \u4e00 -\u9fa5]+',w) == None:
            continue
        else:
            return False
    return True

In [206]:
with open(word_count_file, 'r') as f:
    word_cnt = json.load(f)

In [207]:
total_word_count = {}
for word in word_cnt:
    for day, value in word_cnt[word].iteritems():
        if not total_word_count.has_key(day):
            total_word_count[day] = 0
        total_word_count[day] += value * 1.0 / 10

In [282]:
key_word = u"商品"
save_name = "commodity"
for w, v in model.most_similar(key_word, topn=15):
    if isChinese(w):
        print w, v

商品价格 0.5750259161
大宗 0.572187960148
商品交易 0.491748958826
色系 0.464725375175
从康波 0.439159065485
国内定价 0.435334742069
走势 0.408672451973
商品生产 0.408352673054
概况 0.392053604126
市场走势 0.391019999981
金银铜 0.390180617571


In [283]:
word_df = pd.DataFrame({"date": word_cnt[key_word].keys(), "count": word_cnt[key_word].values()})

In [284]:
word_df.index = word_df["date"].map(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d"))
word_df.sort_index(inplace=True)

In [285]:
word_df

,count,date
date,,
2017-01-12,14,2017-01-12
2017-01-13,81,2017-01-13
2017-01-14,44,2017-01-14
2017-01-15,36,2017-01-15
2017-01-16,126,2017-01-16
2017-01-17,116,2017-01-17
2017-01-18,105,2017-01-18
2017-01-19,170,2017-01-19
2017-01-20,103,2017-01-20


In [286]:
hot_val = {}
threshold = 0.1
for word in word_cnt:
    if isChinese(word) and word in model.vocab:
        dis = model.similarity(word, key_word)
        if dis >= threshold:
            for day, value in word_cnt[word].iteritems():
                if not hot_val.has_key(day):
                    hot_val[day] = 0
                hot_val[day] += value * dis
for key, value in hot_val.iteritems():
    hot_val[key] = value / total_word_count[key]

In [287]:
hot_df = pd.DataFrame({"date": hot_val.keys(), "value": hot_val.values()})

In [288]:
hot_df["value"] = hot_df["value"] * 10000

In [289]:
hot_df.index = hot_df["date"].map(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d"))
hot_df.sort_index(inplace=True)

In [290]:
hot_df

,date,value
date,,
2017-01-12,2017-01-12,1168.429384
2017-01-13,2017-01-13,1565.657473
2017-01-14,2017-01-14,1445.571045
2017-01-15,2017-01-15,1672.667643
2017-01-16,2017-01-16,1736.061555
2017-01-17,2017-01-17,1560.283852
2017-01-18,2017-01-18,1748.775102
2017-01-19,2017-01-19,1723.476771
2017-01-20,2017-01-20,1680.230861


In [291]:
hot_df["date"] = hot_df["date"].map(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d").strftime("%Y%m%d"))

In [292]:
hot_df.to_csv("%s/%s.csv"%(asset_class_dir, save_name), index=False)

In [160]:
df = ts.get_hist_data("sh")

In [162]:
df.index = pd.to_datetime(df.index)

In [163]:
df = df[df.index >= datetime.datetime(2017, 1, 12)]

In [165]:
df = df[["close"]]

In [166]:
df.columns = ["value"]

In [167]:
df["date"] = df.index.map(lambda x: x.strftime("%Y%m%d"))

C:\Users\jgtzsx01\AppData\Local\Continuum\Anaconda2\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [168]:
df

,value,date
date,,
2017-01-24,3142.88,20170124
2017-01-23,3136.77,20170123
2017-01-20,3123.14,20170120
2017-01-19,3101.30,20170119
2017-01-18,3113.01,20170118
2017-01-17,3108.77,20170117
2017-01-16,3103.43,20170116
2017-01-13,3112.76,20170113
2017-01-12,3119.29,20170112


In [169]:
df.to_csv("%s/stock.csv"%(asset_class_dir), index=False)